# **16 Чтение и запись данных. Часть 1 (лекции А. Джумурата)**

Курс ведёт **Александр Джумурат** Data Scientist в ivi

## **16.1** *Чтение текстовых и CSV файлов*

### python text


Если речь идёт о продакшн-системах машинного обучения, то данные могут приходить в форматах, мало пригодных для использования. Аналитику данных часто приходится иметь дело со слабо-структурированным текстом - например, текстовыми логами. Для открытия простых тестовых файлов (например, это может быть access-лог веб-сервера nginx) в python используются [файловые дескрипторы](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files), которые рекомендуется использовать совместно с [менеджером контекста with](https://docs.python.org/3/reference/datamodel.html#context-managers). Файл, открытый на чтение предоставляет интерфейс итератора - а значит, может использоваться внутри цикла for:

In [1]:
file_path = './data/uwsgi.log'

with open(file_path, 'r') as f:
    for row in f:
        print(row)

192.168.101.4 - - [05/Feb/2019:21:36:07 +0300] "GET /movie/is_personalizable/?history_type=watch&uid=5734473158358418&master_uid=5734473158358418 HTTP/1.1" 200 75 "-" "python-requests/2.0.0 CPython/2.7.3 Linux/4.4.0-47-generic"

192.168.102.3 - - [05/Feb/2019:21:36:07 +0300] "POST /logger/content/time/ HTTP/1.1" 404 305 "-" "Mozilla/5.0 (Windows; U; en-US; rv:1.8.1.11; Gecko/20071129; Firefox/2.5.0) Maple 6.0.00067 Navi"

192.168.102.3 - - [05/Feb/2019:21:36:07 +0300] "GET /movie/collection/items/recommendations/?uid=1623029046&master_uid=1623029046&collection_id=1525&subsite=141&app_version=10924&user_ab_bucket=10679 HTTP/1.1" 200 535 "-" "python-requests/2.0.0 CPython/2.7.3 Linux/3.13.0-24-generic"

192.168.7.46 - - [05/Feb/2019:21:36:07 +0300] "GET /movie/recommendations/?uid=803285924&master_uid=803285924&iid=102751&user_ab_bucket=12493&top=30&scenario_id=ITEM_PAGE&app_version=15100&subsite=3021 HTTP/1.1" 200 1392 "-" "python-requests/2.0.0 CPython/2.7.3 Linux/3.13.0-24-generic"




Этот минималисчтичный код открывает большие возможности для процессинга строк - например, с применением [регулярных выражений](https://docs.python.org/3/library/re.html). В процесс чтения файла часто встраивают код для обработки строк. Например, распечатаем IP-адреса, с которых производился доступ

In [2]:
with open(file_path, 'r') as f:
    for row in f:
        # оператор split разделяет строку по пробелам
        parted_row = row.split(' ')
        if len(parted_row)>1:
            print("IP adress: " + parted_row[0])

IP adress: 192.168.101.4
IP adress: 192.168.102.3
IP adress: 192.168.102.3
IP adress: 192.168.7.46


## **16.2-16.3** *CSV-файлы*

Для чтения csv в Python существует два основных способа - [модуль csv](https://docs.python.org/3/library/csv.html), который доступен в стандартной библиотеке и библиотека для обработки табличных данных [pandas](https://pandas.pydata.org/).

### python csv

Базовый примитив в модуле csv - это [reader](https://docs.python.org/3/library/csv.html#csv.reader), который представляет собой итератор по строкам csv-файла. Более интересным объектом является [DictReader](https://docs.python.org/3/library/csv.html#csv.DictReader), который читает каждую строку как словарь, в котором ключом является имя столбца, а значением - элемент строки, находящийся в этом поле.

Проведём разбор csv-файла с помощью xml.DictReader:

In [3]:
import csv

file_path = './data/task.csv'

sample_row = None
with open(file_path, newline='', encoding='utf8') as csvfile:
    reader = csv.DictReader(csvfile)
    print(reader.fieldnames)
    for row in reader:
        print([row[field_name] for field_name in reader.fieldnames])
        sample_row = row

['Код', 'Тема', 'Компонент', 'Затрачено в часах']
['HYDRA-535', 'Пробрасывать пользовательское распределение paid_types в ехидну', 'echidna', '1']
['HYDRA-534', 'Гибридный рекомендатель с multi-channel feedback', 'hydra', '3']
['HYDRA-532', 'Джоба в дженкинсе для расчёта динамики РВП', 'hydramatrices', '2']
['HYDRA-531', 'Интеграция Hydra с Gamora', 'hydramagrices', '4']
['HYDRA-530', 'Тестируем интеграцию с Jira', 'hydra', '2']
['HYDRA-527', 'Поправить функцию _get_ui_rec_matrix', 'hydra', '10']
['HYDRA-524', 'Оптимизировать матрицу ItemFactors', 'hydra', '2']
['HYDRA-523', 'Сортировка ЦПБ', 'hydra', '5']
['HYDRA-520', 'Закостылить параметр top', 'hydra', '2']
['HYDRA-519', "Сделать 'stable' конфигом по умолчанию в Гидре", 'hydra', '2']
['HYDRA-518', 'Неудобно тестировать запись в redis', 'hydramatrices', '1']
['HYDRA-517', 'Улучшить рекомендации (первая итерация)', 'hydra', '7']
['HYDRA-514', 'Добавить логирование в скрипты hydra/bin', 'hydramagrices', '5']
['HYDRA-513', 'Поправить s

In [13]:
sample_row # каждая строка - как словарь OrderedDict

OrderedDict([('Код', 'HYDRA-506'),
             ('Тема', 'Техдолг по логике /collection/recommendations/'),
             ('Компонент', 'hydra'),
             ('Затрачено в часах', '24')])

Каждая строка представляет собой объект формата [OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict)

In [15]:
print(type(row))

<class 'collections.OrderedDict'>


### pandas csv

Основная библиотека python для работы с табличными данными - это pandas. Она поддерживает большое количество трансформаций данных, включая такую экзотику как оконные функции. Ещё в этом модуле "из коробки" доступны функции для расчёта агрегированных статистик и богатые возможности графики.

Единственный способ для чтения файлов - метод [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), который содержит огромное количество параметров и позволяет гибко конфигурировать процесс чтения:

In [6]:
import pandas as pd

file_path = './data/task.csv'

df = pd.read_csv(file_path)
print(type(df))
df.head(10)

<class 'pandas.core.frame.DataFrame'>


,Код,Тема,Компонент,Затрачено в часах
0,HYDRA-535,Пробрасывать пользовательское распределение pa...,echidna,1
1,HYDRA-534,Гибридный рекомендатель с multi-channel feedback,hydra,3
2,HYDRA-532,Джоба в дженкинсе для расчёта динамики РВП,hydramatrices,2
3,HYDRA-531,Интеграция Hydra с Gamora,hydramagrices,4
4,HYDRA-530,Тестируем интеграцию с Jira,hydra,2
5,HYDRA-527,Поправить функцию _get_ui_rec_matrix,hydra,10
6,HYDRA-524,Оптимизировать матрицу ItemFactors,hydra,2
7,HYDRA-523,Сортировка ЦПБ,hydra,5
8,HYDRA-520,Закостылить параметр top,hydra,2
9,HYDRA-519,Сделать 'stable' конфигом по умолчанию в Гидре,hydra,2


Pandas позволяет проводить действия над csv более быстро. Объект DataFrame предоставляет широкие возможности по агрегации данных. например, вычислим среднее время затраценное на задачу внутри каждого компонента

In [7]:
df.groupby(by=['Компонент'])['Затрачено в часах'].mean().to_frame().head()

,Затрачено в часах
Компонент,
echidna,1.000000
hydra,6.545455
hydra_utils,16.000000
hydramagrices,4.000000
hydramatrices,1.666667


В более общем виде можно вызвать метод agg, куда передать агрегирующие функции (например, из numpy)

In [37]:
import numpy as np

# параметры аггрегации в виде dict
agg_config = {
    'Затрачено в часах': {
        'mean_val': np.mean, 'max_val': np.max}
}
# применяем параметры к выборке
df.groupby(
    by=['Компонент']
)[
    ['Затрачено в часах']
].aggregate(
    agg_config
)

,mean_val,max_val
Компонент,,
echidna,1.000000,1
hydra,6.545455,24
hydra_utils,16.000000,16
hydramagrices,4.000000,5
hydramatrices,1.666667,2


## **16.4** *Работа с данными формата HTML и XML*

Для работы с XML и HTML можно использовать сторонний модуль [lxml](https://lxml.de/), который является обёрткой над С-библиотеками  **libxml2** и **libxslt**.

Пример кода, который проводит разбор большого XML, можно посмотреть в проекте по парсингу [отчётов системы аналитики Tableau ](https://github.com/Dju999/TableauParser)

### Работа с HTML

Библиотека [lxml.html](https://lxml.de/lxmlhtml.html) позволяет обращаться к данным html с помощью XPath.

Xpath - это декларативный язык для обращения к html данным. Он позволяет выбирать содержимое веб-страницы, выполняя навигацию по тегам, блокам div и т.д.

Перед тем, как использовать XPath, придётся посмотреть [на html-код страницы](view-source:https://skillbox.ru/code) и найти интересующие нас тэги.

Видно, что нам интересен тэг `menu__name` - названия крупных разделов в каталоге курса

In [71]:
import requests
from lxml import html

page = requests.get('https://skillbox.ru').content.decode('UTF-8')

html_tree = html.fromstring(page)

items = html_tree.xpath("//a[contains(@class, '{}')]".format('ui-tab ui-tab--link ui-tab--main'))


print("Результат применения XPath: ", items)

print("\nСписок разделов:\n")
for item in items:
    print(item.text)

Результат применения XPath:  [<Element a at 0x1aba20b3ef0>, <Element a at 0x1ab9f8d31d0>, <Element a at 0x1ab9f8d3130>, <Element a at 0x1aba27c8770>, <Element a at 0x1aba27c8cc0>]

Список разделов:


        Программирование
      

        Дизайн
      

        Маркетинг
      

        Управление
      

        Игры
      


Видим, что второй блок называется "Программирование" - то, что нужно. Достанем список курсов по программированию. Находим группу курсов, которая соответствуем каталогу "Программирование" со страницы [skillbox.ru/code](https://skillbox.ru/code)

In [19]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

options = webdriver.ChromeOptions()
options.add_argument('headless') # опция для работы в безголовом режиме
driver = webdriver.Chrome(options=options) # загрузка драйврера
wait = WebDriverWait(driver, 15)  # задаем ожидание (н-р до появления элемента)
driver.get('https://skillbox.ru/code')  # переходим на страницу
button = driver.find_element(By.XPATH, '//div[2]/section[1]/button') # поиск кнопки
driver.execute_script("window.scrollTo(0, 2000)") # прокрутка страницы вниз
button.click()
button.click() # два клика по кнопке для развёртывания списка курсов
time.sleep(3) # ожидание перед парсингом 3 сек. чтобы страница загрузилась
page = driver.page_source # загрузка страницы из вебдрайвера
driver.close() # закрываем драйвер

# page = requests.get('https://skillbox.ru/code').content.decode('UTF-8')
html_tree = html.fromstring(page)

items_list = html_tree.xpath("//a[contains(@class, '--profession')]/div/h3[@class='{}']"
                             .format('card__title h h--4'))

# button = html_tree.xpath('//div[2]/section[1]/button/text()')[0]

for item in items_list:
    print(item.text.replace('\u200c', '').strip(' '))

Data Scientist
Python-разработчик
Java-разработчик
Я — Веб-разработчик PRO
Тестировщик
Frontend-разработчик
IOS-разработчик
Тестирование мобильных приложений
Android-разработчик
Специалист по кибербезопасности
1C-разработчик
Разработчик на C++ с нуля
Data Scientist: анализ данных
Java-разработчик PRO
DevOps-инженер PRO
С#-разработчик
Frontend-разработчик PRO
PHP-разработчик
Data Scientist: машинное обучение
Go-разработчик
IT-рекрутёр
DevOps-инженер
Разработчик мобильных приложений
Fullstack-разработчик на Python
Fullstack-разработчик на JavaScript
Fullstack-разработчик на PHP
Frontend-разработчик
Старт в DevOps:
системное администрирование
для начинающих
Веб-разработчик с нуля до PRO


Чтобы получить путь `XPath` до любого элемента, надо:
    
* Открыть в браузере Google Chrome страничку с курсами `https://skillbox.ru/courses/code/`
* Открыть в настройках пункт **Инструменты разработчика**, должна получиться вот такая картинка

Скриншот из инспектора в браузере:
![](https://sun9-13.userapi.com/c858420/v858420287/1ca12d/nPo2S99Fay4.jpg)

* найти в открывшемся списке элемент, до которого вы хотите посмотреть путь **XPath** (он будет подсвечивать на html странице)
* нажать на значок из трёх точек `...` рядом с этим элементом
* из выпавшего списка выбрать пункт **Скопировать XPath**

![xpath](img/xpath.png)

## **16.5** *Работа с данными формата XML*

### Описание формата XML

XML (eXtensible Markup Language) — [расширяемый язык разметки](https://ru.wikipedia.org/wiki/XML) текстовых документов. Организация Консорциум Всемирной паутины (W3C) занимается разработкой [стандарта XML](https://www.w3.org/XML/)

Пример документа в формате XML:
<pre> 
<code>
&lt; ?xml version="1.0" ?&gt;
&lt; contentTitles reminder="15"&gt;
    &lt; content &gt;
        &lt; releaseDate &gt; 1181251680 &lt; /releaseDate &gt;
        &lt; id &gt; 040000008200E000 &lt; /id &gt;
    &lt; /content &gt;
&lt; /contentTitles &gt;
</code>
</pre>

Видно, что файл содержит версионирование ```xml_version="1.0"```, а так же имеет иерархическую структуру. Минимальный логический элемент XML называется **сущностью** (англ. entity). У *сущности* есть имя (тест, заключённый в угловые скобки) и значение (англ. **value**) - всё, что содержится внутри сущности. Одним из подмножеств XML является язык разметки web-страниц *HTML*. Сущности HTML зафиксированы в стандарте html, а сущности XML нигде не зафиксированы - их определяет для себя разработчик в кажом конкретном случае (пример выше - xml-документ, который описывает некоторые свойства контента а базе данных онлайн-кинотеатра, например дату выпуска *releaseDate* и внутренний идентификатор *id*)

Для разбора XML тоже можно использовать XPath

In [7]:
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'

with open(file_path) as f:
    
    doc = ElementTree.parse(f)
    content_titles = doc.getroot()
    for movie in content_titles.findall("./Content/content_title/[genre='drama']"):
        print(movie.find("./title").text)

The Shawshank Redemption
The Dark Knight


## **16.6** *Работа с данными формата XLSX*

Неплохой библиотекой для работы с форматом xlsx является [открытый проект openpyxl](https://openpyxl.readthedocs.io/en/stable/).

Для установки пакета в Anaconda выполните:
<pre>
conda install -c anaconda openpyxl
</pre>


Для примера загрузим программу этого курса. Библиотека оперирует стандартными для экселя примитивами: workbook, worksheet, row, column и позволяет итеративно выполнять обход ячеек.

In [12]:
from openpyxl import load_workbook

file_path = './data/Analytics_Junior.xlsx'

wb = load_workbook(file_path)

print("wb = %s" % type(wb))

first_worksheet = wb.worksheets[0]
first_row = first_worksheet[1]
header = [i.value for i in first_row if i.value]

print("\n\nЧитаем excel построчно:\n")
print(header)
for row in first_worksheet.iter_rows(min_row=2, max_row=12):
    if row[len(header)-1].value:
        print([cell.value for cell in row[:len(header)]])

wb = <class 'openpyxl.workbook.workbook.Workbook'>


Читаем excel построчно:

['Модуль', 'Урок']
['Введение в анализ данных', 'Что такое Data Science?']
[None, 'Процесс анализа данных']
['Введение в Python', 'Почему Python?']
[None, 'Установка окружения']
[None, 'Домашняя работа']
['Библиотека NumPy (ч.1)', 'Основные идеи NumPy']
[None, 'Ndarray - базовая концепция и операции']
[None, 'Базовые операции']
[None, 'Работа с массивами']
[None, 'Условия и булевы массивы']
[None, 'Домашняя работа']


___

# Дополнительное чтение

Кроме мощной бибилиотеку lxml средства для обработки структурированных текстовых файлов есть и в самом Python.

Этот раздел просто для ознакомления, Домашки по нему не будет

## *Альтернативы XPath*

Форматы документов XML и HTML представляют собой древовидные структуры. Кроме XPath для их обработки можно применять подход, свойственный для древовидых структур данных - рекурсивно спуститься вниз от корня к листям и проделать какие-то действия: например, найти нужный тэг и применить внутри него XPath

### python lxml.html

Тут парсинг производится с помощью метода **fromstring**:

In [28]:
import requests
from lxml import html, etree

# hparser = etree.HTMLParser(encoding='utf-8')

page = requests.get('https://skillbox.ru/courses/code').content.decode('utf-8')

tree = html.fromstring(page)

In [29]:
root = tree.getchildren()

cnt = 0
for tag in root:
    for subtag in tag.getchildren():
        if cnt < 10:
            print(subtag.tag)
            print(subtag.text_content())
            cnt += 1
        else:
            pass

title
Skillbox – онлайн-университет, один из лидеров российского рынка онлайн-образования.
meta

meta

meta

meta

meta

meta

meta

meta

meta



## *XML в стандартной библиотеке python*

### xml

В стандартную библиотеку python входит [модуль xml](https://docs.python.org/3.7/library/xml.html), который реализует несколько API для чтения XML - среди них наиболее популярными являются [minidom](https://docs.python.org/3.5/library/xml.dom.minidom.html#module-xml.dom.minidom)(которая является урезанной реализацией [стандарта DOM](https://www.w3.org/DOM/) ) и [ElementTree](https://docs.python.org/3.5/library/xml.etree.elementtree.html#module-xml.etree.ElementTree), которая реализует другой алгоритм чтения и создания xml. В ранних версиях python эти API сильно различались по [спискам существующих в них уязвимостей](https://docs.python.org/3.7/library/xml.html#xml-vulnerabilities), но начиная с версии python3.7 сильной разницы между ними нет (кроме, собственно, методов API).

Для примера произведём разбор одного и того же файла помощью этих двух модулей. Решим простую задачу: обойдём xml и составим документ в формате json соответсвия id контента и даты его длительности.

In [14]:
import xml.dom.minidom
from xml.etree import ElementTree

file_path = './data/xml_content_description.xml'

content_descriptions_minidom = []

with open(file_path) as f:
    
    doc = xml.dom.minidom.parse(f)
    for content_title in doc.getElementsByTagName('content_title'):
        try:
            content_descriptions_minidom += [
                {"title": content_title.getElementsByTagName('title')[0].firstChild.data,
                "duration": content_title.getElementsByTagName('duration')[0].firstChild.data}
            ]
        except IndexError:
            pass

print(content_descriptions_minidom)

[{'title': 'The Shawshank Redemption', 'duration': '8520'}, {'title': 'The Dark Knight', 'duration': '9120'}]
